In [1]:
# Changing the working directory
import os
os.chdir('../')

import gspread
from utils import *
from skydb.sheets import updateSpreadsheet

In [ ]:
records = getOfficialRecords()
grades = records['grades']
gpas = records['gpas']
students = getStudents()
psat_scores = getPsatScores()

In [ ]:
""" PSAT Scores """
psat_students = (
    students
    .query("""grade_level in [9,10,11]""")
    .drop('counselor', axis=1)
    .rename(columns={'id':'user_id'})
    .astype({'user_id':'int64', 'student_id':'float', 'grade_level':'int64'})
    .merge(psat_scores,
                'left',
                on='user_id'
    )
)

In [ ]:
""" Cumulative GPA """
gpa_students = (
    psat_students
    .merge(gpas,
           'inner',
           on='user_id'
    )
)

In [ ]:
""" Cleaning English/History Grades """
raw_english_grades = grades.loc[grades.transcript_category.str.contains('English|History')].copy()
for index, row in raw_english_grades.iterrows():
    if row['school_year'] == '2021 - 2022' and row['term'] == 'Fall':
        raw_english_grades.loc[index, 'official_grade'] = row['grade']
    elif row['term'] == 'Spring':
        raw_english_grades.loc[index, 'official_grade'] = row['grade']
    else:
        raw_english_grades.loc[index, 'official_grade'] = None
        
english_grades = raw_english_grades[~raw_english_grades.official_grade.isna()].copy().drop('grade', axis=1)

""" Adding label for English/History classes"""
english_grades['label'] = (
    english_grades['transcript_category'] +
    (
        english_grades
        .groupby(['user_id', 'transcript_category'])
        .cumcount()
    ).astype('str')
)

english_grades.label = english_grades.label.str.replace(' ', '')

In [ ]:
""" Spread Data """
clean_english_students = (
    english_grades
    .groupby(['user_id', 'label'])
    [['official_grade', 'course_title']]
    .first()
    .unstack()
    .copy()
)

In [ ]:
""" Spread Data """
clean_english_students = (
    english_grades
    .groupby(['user_id', 'label'])
    [['official_grade', 'course_title']]
    .first()
    .unstack()
    .copy()
)

""" Cleaning Column Names """
clean_english_students.columns = (
clean_english_students.columns.get_level_values(1) + " "+ clean_english_students.columns.get_level_values(0)
)

clean_english_students = clean_english_students[[
    'English0 course_title', 'English0 official_grade',
    'History0 course_title', 'History0 official_grade',
    'English1 course_title', 'English1 official_grade',
    'History1 course_title', 'History1 official_grade',
    'English2 course_title', 'English2 official_grade',
    'History2 course_title', 'History2 official_grade',
    'English3 course_title', 'English3 official_grade',
    'History3 course_title', 'History3 official_grade',
    'English4 course_title', 'English4 official_grade',
    'History4 course_title', 'History4 official_grade',
    'History5 course_title', 'History5 official_grade',
    'History6 course_title', 'History6 official_grade',
    'History7 course_title', 'History7 official_grade',
]].reset_index()


In [ ]:
""" Joining with student Data """
english_data = (
    gpa_students
    .merge(clean_english_students,
           'inner',
           on='user_id'
    )
    .sort_values(['last_name', 'first_name'])
    .drop(['user_id', 'student_id', 'first_name'], axis=1)
)

In [ ]:
# English Data
updateSpreadsheet(english_data.fillna(''), 
                  sheet_id='1NAb2aSSwRuytJvwn9NMcz6oaQChFaPBCvD14cUuprRY',
                  styleClass=HysonFireStyle
                 )

In [ ]:
# History Data
updateSpreadsheet(english_data.fillna(''), 
                  sheet_id='1An6cpwBwvdvpF_OpBV5TI74A',
                  styleClass=HysonFireStyle
                 )